In [1]:
import pandas as pd

# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
pd.set_option('display.width', None)        # Let the display adjust to the window

# 2. Set the display width (optional but often helpful)
#    'None' tries to detect terminal width. 
#    A large number (e.g., 1000) ensures no wrapping unless absolutely necessary.
pd.set_option('display.width', 1000) 

In [2]:
# process_files.py
from config import date_str, DOWNLOAD_DIR, DEST_DIR
from pathlib import Path  # Better path handling

print(f"date_str: {date_str}")

date_str: 2025-04-04


In [3]:
import sys
from pathlib import Path

# Notebook cell
%load_ext autoreload
%autoreload 2

# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

# Verify path
print(f"Python will look in these locations:\n{sys.path}")


# --- Execute the processor ---
import utils

SOURCE_PATH_OHLCV = f'..\data\{date_str}_df_OHLCV_clean.parquet'
SOURCE_PATH = f'..\data\{date_str}_df_finviz_merged.parquet'
PARQUET_PATH_COVARIANCE = f'..\data\{date_str}_df_cov_emv_matrix.parquet'
PARQUET_PATH_CORRELATION = f'..\data\{date_str}_df_corr_emv_matrix.parquet'

Python will look in these locations:
['C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\python311.zip', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\DLLs', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\Lib', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv', '', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\Pythonwin', 'c:\\Users\\ping\\Files_win10\\python\\py311\\stocks\\src']


In [4]:
import pandas as pd

df_ohlcv = pd.read_parquet(SOURCE_PATH_OHLCV)
df_finviz = pd.read_parquet(SOURCE_PATH)

print(f"Loaded df_ohlcv shape: {df_ohlcv.shape}")
print(f"Loaded df_finviz shape: {df_finviz.shape}")


Loaded df_ohlcv shape: (386038, 9)
Loaded df_finviz shape: (1538, 55)


In [5]:
# Combine indices from both dataframes and remove duplicates
tickers = list(set(df_finviz.index))
print(f"Total unique tickers: {len(tickers)}")

Total unique tickers: 1538


In [6]:
df_close = df_ohlcv['Adj Close'].unstack(level=0)

# Assume `tickers` is your predefined list of tickers
common_tickers = df_close.columns.intersection(tickers)
df_close = df_close[common_tickers]

# Assuming `tickers` is your original list of tickers
missing_tickers = list(set(tickers) - set(df_close.columns))

# Print the missing tickers (if any)
if missing_tickers:
    print(f"{len(missing_tickers)} tickers not found in df_close:")
    print(missing_tickers)
else:
    print("All tickers in your list are present in df_close.")

print(f"\nShape of filtered df_close: {df_close.shape}")
display(df_close.head())

All tickers in your list are present in df_close.

Shape of filtered df_close: (251, 1538)


Symbol           A     AA    AAL   AAON    AAPL    ABBV    ABEV    ABNB     ABT   ACGL    ACI   ACIW    ACM     ACN    ACWI    ACWV   ACWX    ADBE    ADC     ADI    ADM     ADP    ADSK   ADT    AEE   AEG    AEM    AEP    AER    AES     AFG    AFL   AFRM    AGCO    AGG    AGI  AGNC    AIG     AIT     AIZ     AJG    AKAM   ALAB     ALB    ALC    ALGN    ALK     ALL    ALLE   ALLY    ALNY   ALSN     ALV     AM    AMAT  AMCR     AMD     AME    AMGN    AMH   AMLP     AMP     AMT    AMX    AMZN      AN   ANET   ANGL    ANSS     AON    AOS    APA     APD    APG    APH     APO    APP    APPF   APTV     AR   ARCC     ARE    ARES    ARGX   ARKB   ARKK     ARM   ARMK     AS    ASML    ASND     ASR  ASTS    ASX    ATI     ATO     ATR     AU   AUR     AVB   AVDE   AVDV   AVEM    AVGO   AVLV   AVTR   AVUS   AVUV     AVY     AWI     AWK    AXON     AXP    AXS   AXTA     AYI   AZEK    AZN      AZO      BA   BABA    BAC     BAH   BALL    BAM     BAP    BAX   BBAX   BBCA     BBD   BBEU   BBIN   BBIO  \
Date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
2024-04-05  143.06  36.08  13.76  89.58  168.78  164.02  2.3873  161.77  108.98  90.24  20.49  32.36  96.20  328.93  107.38  101.35  51.60  485.12  53.65  191.58  61.15  239.70  244.74  6.24  70.43  5.81  61.34  80.84  86.68  17.27  123.89  83.68  32.87  118.16  92.81  14.79  8.34  76.08  194.78  177.15  241.97  106.24  71.34  121.27  84.52  317.58  42.48  169.41  130.80  37.23  153.56  81.39  118.59  13.45  206.11  8.77  170.42  181.04  261.96  34.95  44.16  429.23  183.94  18.09  185.07  158.46  74.40  26.95  341.87  316.99  86.05  34.47  232.77  39.29  58.17  113.09  74.79  232.33  74.73  29.29  18.78  118.56  130.69  387.03  67.43  47.12  124.82  31.87  15.34  973.43  149.88  319.46  2.20  10.79  51.73  113.37  140.10  23.27  3.00  175.40  61.40  63.16  56.71  132.33  62.89  25.87  87.50  90.15  214.42  120.58  115.83  309.90  220.70  64.15  33.02  271.30  49.76  66.09  3104.94  183.14  70.14  36.22  144.67  65.61  39.97  169.20  41.51  46.23  65.18  2.5849  56.59  57.63  28.10   
2024-04-08  143.39  35.93  13.90  92.53  167.66  163.83  2.3873  160.27  108.31  89.91  20.42  33.00  96.52  327.74  107.60  101.54  51.84  484.28  54.71  193.48  60.88  241.45  245.89  6.36  71.04  5.94  60.50  81.14  86.32  17.48  124.73  83.41  34.11  117.22  92.72  14.82  8.37  76.21  195.48  178.53  242.14  105.82  69.92  129.36  83.56  320.06  42.58  168.96  131.34  37.41  156.04  81.67  118.83  13.37  207.29  8.72  169.90  179.74  261.24  35.50  44.13  430.18  182.05  18.53  185.19  159.03  74.35  27.01  344.10  317.83  85.49  33.79  231.75  39.57  57.79  114.46  78.05  233.20  76.07  29.97  18.77  120.59  131.67  386.71  71.79  48.10  129.25  32.18  14.52  976.57  148.88  318.56  2.27  10.63  51.79  113.47  138.89  22.88  3.20  180.34  61.75  63.67  57.19  132.00  62.84  25.67  87.53  90.55  214.51  120.28  117.30  311.21  222.34  64.02  32.96  267.25  50.17  66.34  3076.09  181.56  70.18  36.60  145.15  65.75  40.01  170.32  41.15  46.40  65.27  2.6392  56.88  57.94  28.27   


In [7]:
df_close_sorted = df_close.sort_index()
print(f'df_close_sorted shape: {df_close_sorted.shape}')
display(df_close_sorted.head())

# Calculate daily returns
df_close_returns = df_close_sorted.pct_change().dropna()
print(f'df_close_returns shape: {df_close_returns.shape}')
display(df_close_returns.head())


df_close_sorted shape: (251, 1538)


Symbol           A     AA    AAL   AAON    AAPL    ABBV    ABEV    ABNB     ABT   ACGL    ACI   ACIW    ACM     ACN    ACWI    ACWV   ACWX    ADBE    ADC     ADI    ADM     ADP    ADSK   ADT    AEE   AEG    AEM    AEP    AER    AES     AFG    AFL   AFRM    AGCO    AGG    AGI  AGNC    AIG     AIT     AIZ     AJG    AKAM   ALAB     ALB    ALC    ALGN    ALK     ALL    ALLE   ALLY    ALNY   ALSN     ALV     AM    AMAT  AMCR     AMD     AME    AMGN    AMH   AMLP     AMP     AMT    AMX    AMZN      AN   ANET   ANGL    ANSS     AON    AOS    APA     APD    APG    APH     APO    APP    APPF   APTV     AR   ARCC     ARE    ARES    ARGX   ARKB   ARKK     ARM   ARMK     AS    ASML    ASND     ASR  ASTS    ASX    ATI     ATO     ATR     AU   AUR     AVB   AVDE   AVDV   AVEM    AVGO   AVLV   AVTR   AVUS   AVUV     AVY     AWI     AWK    AXON     AXP    AXS   AXTA     AYI   AZEK    AZN      AZO      BA   BABA    BAC     BAH   BALL    BAM     BAP    BAX   BBAX   BBCA     BBD   BBEU   BBIN   BBIO  \
Date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
2024-04-05  143.06  36.08  13.76  89.58  168.78  164.02  2.3873  161.77  108.98  90.24  20.49  32.36  96.20  328.93  107.38  101.35  51.60  485.12  53.65  191.58  61.15  239.70  244.74  6.24  70.43  5.81  61.34  80.84  86.68  17.27  123.89  83.68  32.87  118.16  92.81  14.79  8.34  76.08  194.78  177.15  241.97  106.24  71.34  121.27  84.52  317.58  42.48  169.41  130.80  37.23  153.56  81.39  118.59  13.45  206.11  8.77  170.42  181.04  261.96  34.95  44.16  429.23  183.94  18.09  185.07  158.46  74.40  26.95  341.87  316.99  86.05  34.47  232.77  39.29  58.17  113.09  74.79  232.33  74.73  29.29  18.78  118.56  130.69  387.03  67.43  47.12  124.82  31.87  15.34  973.43  149.88  319.46  2.20  10.79  51.73  113.37  140.10  23.27  3.00  175.40  61.40  63.16  56.71  132.33  62.89  25.87  87.50  90.15  214.42  120.58  115.83  309.90  220.70  64.15  33.02  271.30  49.76  66.09  3104.94  183.14  70.14  36.22  144.67  65.61  39.97  169.20  41.51  46.23  65.18  2.5849  56.59  57.63  28.10   
2024-04-08  143.39  35.93  13.90  92.53  167.66  163.83  2.3873  160.27  108.31  89.91  20.42  33.00  96.52  327.74  107.60  101.54  51.84  484.28  54.71  193.48  60.88  241.45  245.89  6.36  71.04  5.94  60.50  81.14  86.32  17.48  124.73  83.41  34.11  117.22  92.72  14.82  8.37  76.21  195.48  178.53  242.14  105.82  69.92  129.36  83.56  320.06  42.58  168.96  131.34  37.41  156.04  81.67  118.83  13.37  207.29  8.72  169.90  179.74  261.24  35.50  44.13  430.18  182.05  18.53  185.19  159.03  74.35  27.01  344.10  317.83  85.49  33.79  231.75  39.57  57.79  114.46  78.05  233.20  76.07  29.97  18.77  120.59  131.67  386.71  71.79  48.10  129.25  32.18  14.52  976.57  148.88  318.56  2.27  10.63  51.79  113.47  138.89  22.88  3.20  180.34  61.75  63.67  57.19  132.00  62.84  25.67  87.53  90.55  214.51  120.28  117.30  311.21  222.34  64.02  32.96  267.25  50.17  66.34  3076.09  181.56  70.18  36.60  145.15  65.75  40.01  170.32  41.15  46.40  65.27  2.6392  56.88  57.94  28.27   


df_close_returns shape: (250, 1538)


Symbol             A        AA       AAL      AAON      AAPL      ABBV      ABEV      ABNB       ABT      ACGL       ACI      ACIW       ACM       ACN      ACWI      ACWV      ACWX      ADBE       ADC       ADI       ADM       ADP      ADSK       ADT       AEE       AEG       AEM       AEP       AER       AES       AFG       AFL      AFRM      AGCO       AGG       AGI      AGNC       AIG       AIT       AIZ       AJG      AKAM      ALAB       ALB       ALC      ALGN       ALK       ALL      ALLE      ALLY      ALNY      ALSN       ALV        AM      AMAT      AMCR       AMD       AME      AMGN       AMH      AMLP       AMP       AMT       AMX      AMZN        AN      ANET      ANGL      ANSS       AON       AOS       APA       APD       APG       APH       APO       APP      APPF      APTV        AR      ARCC       ARE      ARES      ARGX      ARKB      ARKK       ARM      ARMK        AS      ASML      ASND       ASR      ASTS       ASX       ATI       ATO       ATR        AU  \
Date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
2024-04-08  0.002307 -0.004157  0.010174  0.032931 -0.006636 -0.001158  0.000000 -0.009272 -0.006148 -0.003657 -0.003416  0.019778  0.003326 -0.003618  0.002049  0.001875  0.004651 -0.001732  0.019758  0.009918 -0.004415  0.007301  0.004699  0.019231  0.008661  0.022375 -0.013694  0.003711 -0.004153  0.012160  0.006780 -0.003227  0.037724 -0.007955 -0.000970  0.002028  0.003597  0.001709  0.003594  0.007790  0.000703 -0.003953 -0.019905  0.066711 -0.011358  0.007809  0.002354 -0.002656  0.004128  0.004835  0.016150  0.003440  0.002024 -0.005948  0.005725 -0.005701 -0.003051 -0.007181 -0.002749  0.015737 -0.000679  0.002213 -0.010275  0.024323  0.000648  0.003597 -0.000672  0.002226  0.006523  0.002650 -0.006508 -0.019727 -0.004382  0.007126 -0.006533  0.012114  0.043589  0.003745  0.017931  0.023216 -0.000532  0.017122  0.007499 -0.000827  0.064660  0.020798  0.035491  0.009727 -0.053455  0.003226 -0.006672 -0.002817  0.031818 -0.014829  0.001160  0.000882 -0.008637 -0.016760   
2024-04-09  0.020364  0.013359  0.003597 -0.035556  0.007217  0.002014  0.016630  0.014912  0.017358 -0.035257 -0.001469  0.025152 -0.016162  0.007994  0.001487  0.001773  0.002315  0.017077  0.023031  0.037782  0.011334  0.013667  0.008296  0.014151  0.008868 -0.005051  0.017355  0.009613 -0.011353 -0.000572 -0.018440 -0.023139  0.036939  0.021071  0.003451  0.013495  0.008363 -0.029917 -0.005474 -0.013555 -0.000124  0.016538 -0.022168 -0.019171 -0.013763  0.031838  0.010099 -0.023378  0.007690  0.014969 -0.000897 -0.006857  0.002188 -0.000748  0.006561  0.024083  0.005180  0.001447  0.004287  0.000000 -0.007931 -0.014180  0.024389 -0.007555  0.002592  0.007797 -0.008877  0.003332  0.001162 -0.001384 -0.005732 -0.004143  0.008544 -0.001516 -0.014016 -0.036082 -0.019603 -0.003045  0.037597  0.008675 -0.001066  0.022805 -0.010101 -0.006646 -0.039142  0.010811 -0.018878 -0.009944  0.028926  0.007250  0.032174 -0.010987  0.017621  0.015052 -0.000965  0.002644 -0.001440 -0.003059   
2024-04-10 -0.021940 -0.

In [8]:
cov_emv_matrix, corr_emv_matrix = utils.get_cov_corr_ewm_matrices_chunked(df_close_returns, span=21, return_corr=True, return_cov=True)

print(f'cov_emv_matrix shape: {cov_emv_matrix.shape}')
display(cov_emv_matrix.head())
print(f'corr_emv_matrix shape: {corr_emv_matrix.shape}')
display(corr_emv_matrix.head())  

cov_emv_matrix shape: (1538, 1538)


Symbol         A        AA       AAL      AAON      AAPL      ABBV      ABEV      ABNB       ABT      ACGL       ACI      ACIW       ACM       ACN      ACWI      ACWV      ACWX      ADBE       ADC       ADI       ADM       ADP      ADSK       ADT       AEE       AEG       AEM       AEP       AER       AES       AFG       AFL      AFRM      AGCO       AGG       AGI      AGNC       AIG       AIT       AIZ       AJG      AKAM      ALAB       ALB       ALC      ALGN       ALK       ALL      ALLE      ALLY      ALNY      ALSN       ALV        AM      AMAT      AMCR       AMD       AME      AMGN       AMH      AMLP       AMP       AMT       AMX      AMZN        AN      ANET      ANGL      ANSS       AON       AOS       APA       APD       APG       APH       APO       APP      APPF      APTV        AR      ARCC       ARE      ARES      ARGX      ARKB      ARKK       ARM      ARMK        AS      ASML      ASND       ASR      ASTS       ASX       ATI       ATO       ATR        AU       AUR  \
Symbol                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
A       0.000560  0.000832  0.000501  0.000638  0.000660  0.000360  0.000163  0.000548  0.000243  0.000436  0.000137  0.000500  0.000350  0.000438  0.000424  0.000233  0.000353  0.000438  0.000124  0.000758  0.000381  0.000321  0.000394  0.000286  0.000220  0.000635  0.000355  0.000136  0.000621  0.000574  0.000333  0.000403  0.001090  0.000555 -0.000031  0.000359  0.000251  0.000488  0.000492  0.000458  0.000325  0.000480  0.000775  0.000790  0.000446  0.000365  0.000595  0.000359  0.000320  0.000532  0.000708  0.000566  0.000468  0.000493  0.000606  0.000254  0.000724  0.000497  0.000181  0.000306  0.000430  0.000644  0.000002  0.000110  0.000537  0.000313  0.000862  0.000157  0.000427  0.000209  0.000271  0.001181  0.000451  0.000489  0.000602  0.001003  0.001005  0.000302  0.000561  0.000888  0.000467  0.000483  0.001036  0.000335  0.000237  0.000704  0.000837  0.000454  0.000776  0.000405  0.000369  0.000319  0.000718  0.000605  0.000923  0.000223  0.000348  0.000472  0.001104   
AA      0.000832  0.001805  0.000881  0.001061  0.001161  0.000473  0.000249  0.001037  0.000248  0.000656  0.000153  0.001017  0.000594  0.000734  0.000733  0.000329  0.000597  0.000798  0.000116  0.001265  0.000502  0.000464  0.000770  0.000585  0.000277  0.001091  0.000497  0.000068  0.001088  0.000976  0.000485  0.000621  0.002162  0.001068 -0.000055  0.000556  0.000373  0.000719  0.001002  0.000802  0.000444  0.000793  0.001618  0.001455  0.000575  0.000733  0.001179  0.000581  0.000482  0.001036  0.000937  0.001083  0.000915  0.000842  0.001102  0.000396  0.001388  0.000893  0.000131  0.000452  0.000755  0.001171 -0.000172  0.000104  0.001043  0.000662  0.001594  0.000267  0.000744  0.000309  0.000469  0.002205  0.000756  0.000890  0.001141  0.001933  0.002247  0.000532  0.001088  0.001596  0.000757  0.000807  0.001987  0.000431  0.000591  0.001324  0.001449  0.000788  0.001507  0.000828  0.000629  0.000557  0.001011  0.001192  0.001663  0.000283  0.000505  0.000844  0.001997   


corr_emv_matrix shape: (1538, 1538)


Symbol         A        AA       AAL      AAON      AAPL      ABBV      ABEV      ABNB       ABT      ACGL       ACI      ACIW       ACM       ACN      ACWI      ACWV      ACWX      ADBE       ADC       ADI       ADM       ADP      ADSK       ADT       AEE       AEG       AEM       AEP       AER       AES       AFG       AFL      AFRM      AGCO       AGG       AGI      AGNC       AIG       AIT       AIZ       AJG      AKAM      ALAB       ALB       ALC      ALGN       ALK       ALL      ALLE      ALLY      ALNY      ALSN       ALV        AM      AMAT      AMCR       AMD       AME      AMGN       AMH      AMLP       AMP       AMT       AMX      AMZN        AN      ANET      ANGL      ANSS       AON       AOS       APA       APD       APG       APH       APO       APP      APPF      APTV        AR      ARCC       ARE      ARES      ARGX      ARKB      ARKK       ARM      ARMK        AS      ASML      ASND       ASR      ASTS       ASX       ATI       ATO       ATR        AU       AUR  \
Symbol                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
A       1.000000  0.828124  0.599274  0.718158  0.839165  0.654354  0.340094  0.735038  0.519963  0.677845  0.291068  0.710226  0.818388  0.752489  0.884490  0.682712  0.797091  0.665475  0.458660  0.893236  0.567594  0.672647  0.784580  0.586954  0.495015  0.861191  0.562153  0.333885  0.848153  0.709858  0.671816  0.654221  0.707783  0.646859 -0.486778  0.512151  0.718660  0.749064  0.700213  0.733889  0.599864  0.721098  0.614201  0.809426  0.720375  0.627945  0.695619  0.548536  0.757692  0.671246  0.686313  0.786060  0.738833  0.726798  0.852748  0.671892  0.790231  0.877312  0.432608  0.758911  0.810366  0.869632  0.004500  0.203217  0.750497  0.528447  0.812339  0.827934  0.789149  0.598827  0.756766  0.847577  0.839648  0.800998  0.788262  0.855882  0.512682  0.618449  0.770413  0.798426  0.832870  0.840327  0.833319  0.562365  0.320278  0.777814  0.799270  0.736780  0.741286  0.688247  0.533188  0.539074  0.541474  0.759138  0.867092  0.584504  0.773068  0.608141  0.685417   
AA      0.828124  1.000000  0.586182  0.665131  0.821637  0.478750  0.288634  0.774304  0.295754  0.567905  0.181195  0.804903  0.772137  0.702067  0.850991  0.536524  0.750550  0.675324  0.238359  0.830251  0.416440  0.542428  0.853849  0.667137  0.346540  0.824171  0.437881  0.092648  0.827637  0.672641  0.545610  0.560849  0.782034  0.692578 -0.478611  0.441920  0.595191  0.614913  0.794755  0.716205  0.457070  0.664057  0.714012  0.830557  0.516692  0.701735  0.767758  0.495059  0.635768  0.727090  0.506021  0.837621  0.805028  0.690999  0.863258  0.583519  0.843627  0.877136  0.174913  0.624206  0.791348  0.879780 -0.207940  0.106544  0.811225  0.622872  0.836725  0.786438  0.765215  0.492187  0.727293  0.881371  0.783248  0.811957  0.831939  0.918097  0.638125  0.606108  0.831782  0.798663  0.751245  0.781810  0.890093  0.402433  0.443933  0.814690  0.770652  0.712671  0.801309  0.784165  0.506035  0.523051  0.424386  0.833036  0.869556  0.412438  0.624488  0.604788  0.689859   


In [ ]:
cov_emv_matrix.to_parquet(PARQUET_PATH_COVARIANCE, engine='pyarrow', compression='zstd')
print(f'cov_emv_matrix parqueted to {PARQUET_PATH_COVARIANCE}')
display(cov_emv_matrix.info())

corr_emv_matrix.to_parquet(PARQUET_PATH_CORRELATION, engine='pyarrow', compression='zstd')
print(f'corr_emv_matrix parqueted to {PARQUET_PATH_CORRELATION}')
display(corr_emv_matrix.info())

cov_emv_matrix parqueted to ..\data\2025-04-04_df_cov_emv_matrix.parquet
<class 'pandas.core.frame.DataFrame'>
Index: 1538 entries, A to ZTS
Columns: 1538 entries, A to ZTS
dtypes: float64(1538)
memory usage: 18.1+ MB


None

corr_emv_matrix parqueted to ..\data\2025-04-04_df_corr_emv_matrix.parquet
<class 'pandas.core.frame.DataFrame'>
Index: 1538 entries, A to ZTS
Columns: 1538 entries, A to ZTS
dtypes: float64(1538)
memory usage: 18.1+ MB


None